# 1. Hosting GenAI Models
This Notebook is used to host 2 models to SageMaker Endpoints. The first model is Segment Anything Model (SAM) and the second model is the InPainting model.

## 1.1 Install Dependencies and Import Libraries
Here we are using `SageMaker, Boto3` libraries for Python

In [ ]:
# Import libraries
import sagemaker, subprocess, boto3
from datetime import datetime
from sagemaker import s3, get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [ ]:
# Get the right credentials, role and client for SageMaker
sm_client = boto3.client(service_name="sagemaker")
role = get_execution_role()
print(f'Role: {role}')

INSTANCE_TYPE = 'ml.g5.2xlarge'

bashCommand = "tar -cpzf  code.tar.gz code/"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
sts = boto3.client('sts')
AWS_ACCOUNT_ID = sts.get_caller_identity()["Account"]
REGION = s3_client.meta.region_name

bucket = f'gai-inpainting-s3'
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    if 'gai-inpainting-s3' in bucket["Name"]:
        bucket = bucket["Name"]
        break

print(f'Bucket: {bucket}')
sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])

## 1.2 Model 1: SAM

### Create and Host SAM Model
In this section, we will demonstrate how to create and host the endpoint for SAM model.

In [ ]:
SAM_ENDPOINT_NAME = 'sam-pytorch-' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))
%store SAM_ENDPOINT_NAME

prefix_sam = "SAM/demo-custom-endpoint"

model_data_sam = s3.S3Uploader.upload("code.tar.gz", f's3://{bucket}/{prefix_sam}')
print(f'Model Data: {model_data_sam}')

model_sam = PyTorchModel(entry_point='inference_sam.py',
                     model_data=model_data_sam, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'2000000000', 'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '300'},
                     sagemaker_session=sess,
                     name='model-'+SAM_ENDPOINT_NAME)

print(f'SAM Endpoint Name: {SAM_ENDPOINT_NAME}')

predictor_sam = model_sam.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         deserializers=JSONDeserializer(),
                         endpoint_name=SAM_ENDPOINT_NAME)

## 1.3 Model 2: InPainting

### Create and Host InPainting Model
In this section, we will demonstrate how to create and host the endpoint for InPainting model.

In [ ]:
INPAINTING_ENDPOINT_NAME = 'inpainting-pytorch-' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))
%store INPAINTING_ENDPOINT_NAME

prefix_inpainting = "InPainting/demo-custom-endpoint"

model_data_inpainting = s3.S3Uploader.upload("code.tar.gz", f"s3://{bucket}/{prefix_inpainting}")
print(f'Model Data: {model_data_inpainting}')

model_inpainting = PyTorchModel(entry_point='inference_inpainting.py',
                     model_data=model_data_inpainting, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'2000000000', 'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '300'},
                     sagemaker_session=sess,
                     name='model-'+INPAINTING_ENDPOINT_NAME)

print(f'InPainting Endpoint Name: {INPAINTING_ENDPOINT_NAME}')

predictor_inpainting = model_inpainting.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         serializer=JSONSerializer(),
                         deserializers=JSONDeserializer(),
                         endpoint_name=INPAINTING_ENDPOINT_NAME,
                         volume_size=128)